In [12]:
import shapely
from shapely import Polygon

from osgeo import gdal
import rasterio

In [13]:
from rasterio.features import rasterize
from rasterstats import zonal_stats

In [14]:
import atlite
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr

In [15]:
from scipy.constants import physical_constants

In [16]:
weather_excel_path = "Parameters/weather_parameters.xlsx"

weather_parameters = pd.read_excel(weather_excel_path,
                                       index_col = 'Parameters'
                                       ).squeeze('columns')
weather_filename = weather_parameters['Filename']

hexagons = gpd.read_file('Resources/hex_transport.geojson')

cutout = atlite.Cutout('Cutouts/' + weather_filename +'.nc')
layout = cutout.uniform_layout()

In [17]:
hexagons.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World: Afghanistan, Albania, Algeria, American Samoa, Andorra, Angola, Anguilla, Antarctica, Antigua and Barbuda, Argentina, Armenia, Aruba, Australia, Austria, Azerbaijan, Bahamas, Bahrain, Bangladesh, Barbados, Belgium, Belgium, Belize, Benin, Bermuda, Bhutan, Bolivia, Bonaire, Saint Eustasius and Saba, Bosnia and Herzegovina, Botswana, Bouvet Island, Brazil, British Indian Ocean Territory, British Virgin Islands, Brunei Darussalam, Bulgaria, Burkina Faso, Burundi, Cambodia, Cameroon, Canada, Cape Verde, Cayman Islands, Central African Republic, Chad, Chile, China, Christmas Island, Cocos (Keeling) Islands, Comoros, Congo, Cook Islands, Costa Rica, Côte d'Ivoire (Ivory Coast), Croatia, Cuba, Curacao, Cyprus, Czechia, Denmark, Djibouti, Dominica, Dominican Republic, East Timor, Ecuador, Egypt, El Salvador, Equatoria

In [6]:
pv_profile = cutout.pv(
        panel= 'CSi',
        orientation='latitude_optimal',
        layout = layout,
        shapes = hexagons,
        per_unit = True
        )

[########################################] | 100% Completed | 80.46 s


In [7]:
pv_profile

<xarray.DataArray (time: 8784, dim_0: 164)> Size: 12MB
array([[0.        , 0.        , 0.        , ..., 0.00823901, 0.        ,
        0.        ],
       [0.09806071, 0.03523507, 0.11699666, ..., 0.05059279, 0.17309401,
        0.06723595],
       [0.27406973, 0.14849543, 0.33224089, ..., 0.1806548 , 0.35955773,
        0.22924473],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])
Coordinates:
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00
  * dim_0    (dim_0) int64 1kB 0 1 2 3 4 5 6 7 ... 157 158 159 160 161 162 163
Attributes:
    units:    p.u.

In [8]:
# hexagons_hydro = gpd.read_file('Data/Laos_inc_hydro_hex.geojson')
# fig, ax = plt.subplots(figsize=(10, 10))
# hexagons_hydro.plot(ax=ax, edgecolor='k', cmap='viridis')
hexagons

h3_index   n0  n1  n2   n3  n4   n5  waterbody_dist  \
0    846496bffffffff  102  93  38   72  89  110             0.0   
1    8440655ffffffff   19  45  33    9   0    0             0.0   
2    84414d3ffffffff   71   6  40    5  10   78             0.0   
3    8440659ffffffff   27  26  19  108  45    0             0.0   
4    8440659ffffffff   27  26  19  108  45    0             0.0   
..               ...  ...  ..  ..  ...  ..  ...             ...   
159  84414e1ffffffff   60  47  96    0   0    0             0.0   
160  8441653ffffffff  103  52  25   65   0    0             0.0   
161  8441653ffffffff  103  52  25   65   0    0             0.0   
162  84416c3ffffffff   18  90  74    0   0    0             0.0   
163  8441489ffffffff    7  35  12   79  42    0             0.0   

     waterway_dist  road_dist  index  theo_turbines  theo_pv  index_right  \
0              0.0        0.0      0           1753     9831           92   
1              0.0        0.0      1           1767     9547           92   
2              0.0        0.0      2           1811     9665           92   
3              0.0        0.0      3           1740     9430           92   
4              0.0        0.0      3           1740     9430          139   
..             ...        ...    ...            ...      ...          ...   
159            0.0        0.0    112           1232     6722           92   
160            0.0        0.0    113           1324     7132           94   
161            0.0        0.0    113           1324     7132           92   
162            0.0        0.0    114           1777     5442           92   
163            0.0        0.0    115           1748     9182           92   

    country  Vientiane road construction costs  \
0      Laos                                0.0   
1      Laos                                0.0   
2      Laos                                0.0   
3      Laos                                0.0   
4     Other                                0.0   
..      ...                                ...   
159    Laos                                0.0   
160   Other                                0.0   
161    Laos                                0.0   
162    Laos                                0.0   
163    Laos                                0.0   

     Vientiane trucking transport and conversion costs  \
0                                             0.535086   
1                                             0.618337   
2                                             0.437632   
3                                             0.576556   
4                                             0.576556   
..                                                 ...   
159                                           0.553865   
160                                           0.667232   
161                                           0.667232   
162                                           0.552178   
163                                           0.491101   

    Vientiane trucking state  \
0                        NH3   
1                        NH3   
2                        NH3   
3                        NH3   
4                        NH3   
..                       ...   
159                      NH3   
160                      NH3   
161                      NH3   
162                      NH3   
163                      NH3   

     Vientiane pipeline transport and conversion costs  \
0                                             0.447230   
1                                             0.486964   
2                                             0.400684   
3                                             0.467007   
4                                             0.467007   
..                                                 ...   
159                                           0.456694   
160                                           0.510335   
161                                           0.510335   
162                

In [9]:
layout

<xarray.DataArray (y: 35, x: 30)> Size: 4kB
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])
Coordinates:
  * y        (y) float64 280B 14.0 14.25 14.5 14.75 ... 21.75 22.0 22.25 22.5
  * x        (x) float64 240B 100.2 100.5 100.8 101.0 ... 107.0 107.2 107.5

In [10]:
cutout.data

<xarray.Dataset> Size: 886MB
Dimensions:           (x: 30, y: 35, time: 8784)
Coordinates:
  * x                 (x) float64 240B 100.2 100.5 100.8 ... 107.0 107.2 107.5
  * y                 (y) float64 280B 14.0 14.25 14.5 14.75 ... 22.0 22.25 22.5
  * time              (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23...
    lon               (x) float64 240B dask.array<chunksize=(30,), meta=np.ndarray>
    lat               (y) float64 280B dask.array<chunksize=(35,), meta=np.ndarray>
Data variables: (12/13)
    height            (y, x) float64 8kB dask.array<chunksize=(35, 30), meta=np.ndarray>
    wnd100m           (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    wnd_azimuth       (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    roughness         (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    influx_toa        (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    influx_direct     (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    ...                ...
    albedo            (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    solar_altitude    (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    solar_azimuth     (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    temperature       (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    soil temperature  (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    runoff            (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
Attributes:
    module:             era5
    prepared_features:  ['height', 'runoff', 'wind', 'influx', 'temperature']
    chunksize_time:     100
    Conventions:        CF-1.6
    history:            2024-04-28 19:27:54 GMT by grib_to_netcdf-2.28.1: /op...

In [11]:
pv_profile.dim_0.data

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163], dtype=int64)

In [12]:
pv_profile.sel(dim_0=100).max()

<xarray.DataArray ()> Size: 8B
array(0.75825574)
Coordinates:
    dim_0    int64 8B 100

In [18]:
location_hydro = gpd.read_file('Data/hydropower_dams.gpkg')
location_hydro.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}, inplace=True)
location_hydro.rename(columns={'head_example':'head'},inplace=True)
location_hydro

lat         lon status   COD  capacity  head  \
0   13.943893  105.955782   COMM  2020   260.000    20   
1   15.543679  106.260687   COMM  1999   152.100    20   
2   15.774865  107.054896   COMM  2023    60.000    20   
3   15.354431  106.161330   COMM  2021     5.000    20   
4   15.356495  106.498949   COMM  2015    84.800    20   
..        ...         ...    ...   ...       ...   ...   
62  15.491744  106.278621   COMM  1994    45.000    20   
63  15.403775  106.280332   COMM  2009    76.000    20   
64  20.346323  104.381276   COMM  2019     9.136    20   
65  19.977092  103.889435   COMM  2019    32.000    20   
66  20.346313  104.381284   COMM  2019     9.100    20   

                      geometry  
0   POINT (105.95578 13.94389)  
1   POINT (106.26069 15.54368)  
2   POINT (107.05490 15.77487)  
3   POINT (106.16133 15.35443)  
4   POINT (106.49895 15.35650)  
..                         ...  
62  POINT (106.27862 15.49174)  
63  POINT (106.28033 15.40377)  
64  POINT (104.38128 20.34632)  
65  POINT (103.88944 19.97709)  
66  POINT (104.38128 20.34631)  

[67 rows x 7 columns]

In [19]:
hexagons['hydro'] = hexagons['hydro'].fillna(0)
hexagons['hydro'].unique()
# hex_final['capacity'] = hex_final['capacity'].fillna(0)

array([0.000e+00, 4.270e+02, 1.600e+02, 2.000e+01, 6.000e+01, 3.200e+01,
       3.750e+01, 1.680e+01, 5.000e+00, 1.160e+02, 8.600e+01, 2.400e+02,
       6.600e+01, 1.020e+00, 5.600e+00, 2.900e+02, 6.150e+02, 1.500e+01,
       1.450e+01, 3.000e+01, 4.760e+01, 6.900e+01, 5.400e+01, 9.100e+00,
       6.500e+02, 1.900e+02, 7.600e+01, 1.300e+02, 1.800e+01, 1.500e+02,
       1.800e+02, 1.018e+02, 7.000e+01, 4.500e+01, 8.000e+01, 1.920e+01,
       4.000e+01, 1.200e+02, 9.000e+01, 2.750e+02, 1.480e+01, 5.300e+01,
       1.521e+02, 1.075e+03, 6.400e+01])

In [14]:
location_hydro.capacity.max()

1285.0

In [20]:
laos_hydrobasins = gpd.read_file('hydrobasins_lvl10/hybas_as_lev10_v1c.shp')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x
laos_hydrobasins.columns

Index(['HYBAS_ID', 'NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS', 'DIST_SINK',
       'DIST_MAIN', 'SUB_AREA', 'UP_AREA', 'PFAF_ID', 'ENDO', 'COAST', 'ORDER',
       'SORT', 'geometry', 'lat', 'lon'],
      dtype='object')

In [21]:
laos_hydrobasins.to_file('hydrobasins_lvl10/hybas_as_lev10_v1c.gpkg', driver='GPKG')

In [22]:
laos_hydrobasins = gpd.read_file('hydrobasins_lvl10/hybas_as_lev10_v1c.gpkg')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x
laos_hydrobasins.columns

Index(['HYBAS_ID', 'NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS', 'DIST_SINK',
       'DIST_MAIN', 'SUB_AREA', 'UP_AREA', 'PFAF_ID', 'ENDO', 'COAST', 'ORDER',
       'SORT', 'lat', 'lon', 'geometry'],
      dtype='object')

In [23]:
hydro_profile = cutout.hydro(
    plants=location_hydro,
    hydrobasins= laos_hydrobasins,
    per_unit=True                    # Normalize output per unit area
)

Determine upstream basins per plant: 0it [00:00, ?it/s]

Determine upstream basins per plant: 67it [00:02, 25.25it/s]
c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


[########################################] | 100% Completed | 13.76 s


Shift and aggregate runoff by plant: 67it [00:04, 15.73it/s]


In [19]:
hydro_profile

<xarray.DataArray (plant: 67, time: 8784)> Size: 5MB
array([[2.62211042e+08, 2.66023716e+08, 2.61003815e+08, ...,
        2.64499822e+08, 2.59488087e+08, 2.61326647e+08],
       [4.82567967e+05, 4.83096885e+05, 4.82898397e+05, ...,
        4.71429292e+05, 4.71429292e+05, 4.71429292e+05],
       [2.14390377e+05, 2.16412928e+05, 2.14390377e+05, ...,
        1.44295945e+05, 1.44295945e+05, 1.44295945e+05],
       ...,
       [2.02123996e+05, 1.99300599e+05, 2.00618397e+05, ...,
        2.07845994e+05, 2.09425794e+05, 2.09421183e+05],
       [3.44583073e+05, 3.50646255e+05, 3.69371878e+05, ...,
        3.49532449e+05, 3.65601252e+05, 3.57780938e+05],
       [2.02123996e+05, 1.99300599e+05, 2.00618397e+05, ...,
        2.07845994e+05, 2.09425794e+05, 2.09421183e+05]])
Coordinates:
  * plant    (plant) int64 536B 0 1 2 3 4 5 6 7 8 ... 58 59 60 61 62 63 64 65 66
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [86]:
hydro_profile.sel(plant=0).min()

<xarray.DataArray ()> Size: 8B
array(42858776.62495468)
Coordinates:
    plant    int64 8B 0

In [62]:
location_hydro.iloc[1,1]

106.260687

In [21]:
hydro_profile.dims[1]

'time'

## Match annual hydroprofile with capacity of hydropower
### 1 Equation hydropower potential

In [32]:
location_hydro

lat         lon status   COD  capacity  head  \
0   13.943893  105.955782   COMM  2020   260.000    20   
1   15.543679  106.260687   COMM  1999   152.100    20   
2   15.774865  107.054896   COMM  2023    60.000    20   
3   15.354431  106.161330   COMM  2021     5.000    20   
4   15.356495  106.498949   COMM  2015    84.800    20   
..        ...         ...    ...   ...       ...   ...   
62  15.491744  106.278621   COMM  1994    45.000    20   
63  15.403775  106.280332   COMM  2009    76.000    20   
64  20.346323  104.381276   COMM  2019     9.136    20   
65  19.977092  103.889435   COMM  2019    32.000    20   
66  20.346313  104.381284   COMM  2019     9.100    20   

                      geometry  
0   POINT (105.95578 13.94389)  
1   POINT (106.26069 15.54368)  
2   POINT (107.05490 15.77487)  
3   POINT (106.16133 15.35443)  
4   POINT (106.49895 15.35650)  
..                         ...  
62  POINT (106.27862 15.49174)  
63  POINT (106.28033 15.40377)  
64  POINT (104.38128 20.34632)  
65  POINT (103.88944 19.97709)  
66  POINT (104.38128 20.34631)  

[67 rows x 7 columns]

In [31]:
hydro_profile

<xarray.DataArray (plant: 67, time: 8784)> Size: 5MB
array([[2.62211042e+08, 2.66023716e+08, 2.61003815e+08, ...,
        2.64499822e+08, 2.59488087e+08, 2.61326647e+08],
       [4.82567967e+05, 4.83096885e+05, 4.82898397e+05, ...,
        4.71429292e+05, 4.71429292e+05, 4.71429292e+05],
       [2.14390377e+05, 2.16412928e+05, 2.14390377e+05, ...,
        1.44295945e+05, 1.44295945e+05, 1.44295945e+05],
       ...,
       [2.02123996e+05, 1.99300599e+05, 2.00618397e+05, ...,
        2.07845994e+05, 2.09425794e+05, 2.09421183e+05],
       [3.44583073e+05, 3.50646255e+05, 3.69371878e+05, ...,
        3.49532449e+05, 3.65601252e+05, 3.57780938e+05],
       [2.02123996e+05, 1.99300599e+05, 2.00618397e+05, ...,
        2.07845994e+05, 2.09425794e+05, 2.09421183e+05]])
Coordinates:
  * plant    (plant) int64 536B 0 1 2 3 4 5 6 7 8 ... 58 59 60 61 62 63 64 65 66
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [107]:
def hydropower_potential(eta,flowrate,head):
    '''
    Calculate hydropower potential in Megawatts
    eta: Efficiency
    '''
    rho = 997 # kg/m3; Density of water
    g = physical_constants['standard acceleration of gravity'][0] # m/s2; Based on the CODATA constants 2018
    Q = flowrate / 3600 # transform flowrate per h into flowrate per second
    return (eta * rho * g * Q * head) / (1000 * 1000) # MW


eta = 0.75 # Value for system efficiency of hydropower - source
def hydropower_potential_wrapper(flowrate, head):
    return hydropower_potential(eta, flowrate, head)

result = xr.apply_ufunc(
    hydropower_potential_wrapper,
    hydro_profile,
    xr.DataArray(location_hydro['head'], dims=['plant']),  # Ensure head values align with plant dimension
    vectorize=True
)

### 2 Absolute values to capacity factor
#### Change xarray to percentages (If potential > max_capacity: max_capacity ! potential)

In [108]:

# Define the wrapper function to include capacity limiting and capacity factor calculation
def hydropower_potential_with_capacity(flowrate, head, capacity, eta):
    potential = hydropower_potential(flowrate, head, eta)
    limited_potential = xr.where(potential > capacity, capacity, potential)
    capacity_factor = limited_potential / capacity
    return capacity_factor

# Assuming eta is a constant value, flowrate is an xarray.DataArray, and head is a GeoDataFrame
eta = 0.9  # Example efficiency value
head = location_hydro['head'].values  # Extract head values
capacity = location_hydro['capacity'].values  # Extract capacity values

# Create DataArrays for head and capacity to align with the plant dimension
head_da = xr.DataArray(head, dims=['plant'])
capacity_da = xr.DataArray(capacity, dims=['plant'])

# Apply the wrapper function using apply_ufunc
capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    hydro_profile,
    head_da,
    capacity_da,
    eta,
    vectorize=True,
    dask='parallelized',  # Optionally, use Dask for parallel computation
    output_dtypes=[float]
)


In [109]:
capacity_factor

<xarray.DataArray (plant: 67, time: 8784)> Size: 5MB
array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.15510118, 0.15527118, 0.15520739, ..., 0.15152113, 0.15152113,
        0.15152113],
       [0.17467867, 0.17632658, 0.17467867, ..., 0.11756789, 0.11756789,
        0.11756789],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.52641687, 0.53567955, 0.56428653, ..., 0.53397799, 0.55852618,
        0.54657915],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])
Coordinates:
  * plant    (plant) int64 536B 0 1 2 3 4 5 6 7 8 ... 58 59 60 61 62 63 64 65 66
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [92]:
# Check if all values for each plant are equal to 1 across the time dimension
all_ones_per_plant = (capacity_factor == 1).all(dim='time')

# Count the number of plants that have all values equal to 1
count_all_ones = all_ones_per_plant.sum().item()

# Print the result
print(f"Number of plants with all values equal to 1: {count_all_ones}")


Number of plants with all values equal to 1: 3


### 3 Time-series for each Hexagon
#### Hydro plant capacity factor from plants to hexagons

In [29]:
location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)

# Ensure both GeoDataFrames are in the same CRS
if location_hydro.crs != hexagons.crs:
    location_hydro = location_hydro.to_crs(hexagons.crs)

# Perform spatial join to map each hydro plant to its corresponding hexagon
hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')

# Add the hydro plant indices to the mapping
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index

# Get the number of hexagons and time steps dynamically
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

# Initialize an empty DataArray for the new capacity_factor with dynamic dimensions
capacity_hexagon = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

# Iterate through each hexagon and calculate the average capacity factor for plants within it
for hex_index in range(num_hexagons):
    # Find the plant indices that are within the current hexagon
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    
    if len(plants_in_hex) > 0:
        # Select the capacity_factor and capacity values for the plants in the current hexagon
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        plant_capacities = xr.DataArray(location_hydro.set_index('plant').loc[plants_in_hex]['capacity'], dims=['plant'])

        # Calculate the weighted average capacity factor for the current hexagon
        weights = plant_capacities / plant_capacities.sum()
        weighted_avg_capacity_factor = (hex_capacity_factor * weights).sum(dim='plant')
        
        # Assign the calculated weighted average to the new DataArray
        capacity_hexagon.loc[hex_index] = weighted_avg_capacity_factor


NameError: name 'capacity_factor' is not defined

In [116]:
# Check if all values for each plant are equal to 1 across the time dimension
all_zeros_per_plant = (capacity_hexagon == 1).all(dim='time')

# Count the number of plants that have all values equal to 1
count_all_zeros = all_zeros_per_plant.sum().item()

# Print the result
print(f"Number of plants with all values equal to 1: {count_all_zeros}")


Number of plants with all values equal to 1: 2


In [117]:
capacity_hexagon

<xarray.DataArray (hexagon: 808, time: 8784)> Size: 57MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * hexagon  (hexagon) int32 3kB 0 1 2 3 4 5 6 7 ... 801 802 803 804 805 806 807
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [23]:
hexagons

h3_index   n0  n1  n2   n3  n4   n5  waterbody_dist  \
0    846496bffffffff  102  93  38   72  89  110             0.0   
1    8440655ffffffff   19  45  33    9   0    0             0.0   
2    84414d3ffffffff   71   6  40    5  10   78             0.0   
3    8440659ffffffff   27  26  19  108  45    0             0.0   
4    8440659ffffffff   27  26  19  108  45    0             0.0   
..               ...  ...  ..  ..  ...  ..  ...             ...   
159  84414e1ffffffff   60  47  96    0   0    0             0.0   
160  8441653ffffffff  103  52  25   65   0    0             0.0   
161  8441653ffffffff  103  52  25   65   0    0             0.0   
162  84416c3ffffffff   18  90  74    0   0    0             0.0   
163  8441489ffffffff    7  35  12   79  42    0             0.0   

     waterway_dist  road_dist  index  theo_turbines  theo_pv  index_right  \
0              0.0        0.0      0           1753     9831           92   
1              0.0        0.0      1           1767     9547           92   
2              0.0        0.0      2           1811     9665           92   
3              0.0        0.0      3           1740     9430           92   
4              0.0        0.0      3           1740     9430          139   
..             ...        ...    ...            ...      ...          ...   
159            0.0        0.0    112           1232     6722           92   
160            0.0        0.0    113           1324     7132           94   
161            0.0        0.0    113           1324     7132           92   
162            0.0        0.0    114           1777     5442           92   
163            0.0        0.0    115           1748     9182           92   

    country  Vientiane road construction costs  \
0      Laos                                0.0   
1      Laos                                0.0   
2      Laos                                0.0   
3      Laos                                0.0   
4     Other                                0.0   
..      ...                                ...   
159    Laos                                0.0   
160   Other                                0.0   
161    Laos                                0.0   
162    Laos                                0.0   
163    Laos                                0.0   

     Vientiane trucking transport and conversion costs  \
0                                             0.535086   
1                                             0.618337   
2                                             0.437632   
3                                             0.576556   
4                                             0.576556   
..                                                 ...   
159                                           0.553865   
160                                           0.667232   
161                                           0.667232   
162                                           0.552178   
163                                           0.491101   

    Vientiane trucking state  \
0                        NH3   
1                        NH3   
2                        NH3   
3                        NH3   
4                        NH3   
..                       ...   
159                      NH3   
160                      NH3   
161                      NH3   
162                      NH3   
163                      NH3   

     Vientiane pipeline transport and conversion costs  \
0                                             0.447230   
1                                             0.486964   
2                                             0.400684   
3                                             0.467007   
4                                             0.467007   
..                                                 ...   
159                                           0.456694   
160                                           0.510335   
161                                           0.510335   
162                

In [31]:
location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)

# Ensure both GeoDataFrames are in the same CRS
if location_hydro.crs != hexagons.crs:
    location_hydro = location_hydro.to_crs(hexagons.crs)



# Perform spatial join to map each hydro plant to its corresponding hexagon
hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')

# Add the hydro plant indices to the mapping
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index

# Get the number of hexagons and time steps dynamically
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

# Initialize an empty DataArray for the new capacity_factor with dynamic dimensions
capacity_hexagon = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

In [29]:
hydro_hex_mapping

lat         lon status   COD  capacity  head  \
0   13.943893  105.955782   COMM  2020   260.000    20   
0   13.943893  105.955782   COMM  2020   260.000    20   
1   15.543679  106.260687   COMM  1999   152.100    20   
2   15.774865  107.054896   COMM  2023    60.000    20   
2   15.774865  107.054896   COMM  2023    60.000    20   
..        ...         ...    ...   ...       ...   ...   
62  15.491744  106.278621   COMM  1994    45.000    20   
63  15.403775  106.280332   COMM  2009    76.000    20   
64  20.346323  104.381276   COMM  2019     9.136    20   
65  19.977092  103.889435   COMM  2019    32.000    20   
66  20.346313  104.381284   COMM  2019     9.100    20   

                      geometry  index_right         h3_index   n0  ...  index  \
0   POINT (105.95578 13.94389)          156  846590bffffffff   61  ...    111   
0   POINT (105.95578 13.94389)          157  846590bffffffff   61  ...    111   
1   POINT (106.26069 15.54368)           97  8465921ffffffff   30  ...     69   
2   POINT (107.05490 15.77487)          160  8441653ffffffff  103  ...    113   
2   POINT (107.05490 15.77487)          161  8441653ffffffff  103  ...    113   
..                         ...          ...              ...  ...  ...    ...   
62  POINT (106.27862 15.49174)           97  8465921ffffffff   30  ...     69   
63  POINT (106.28033 15.40377)           97  8465921ffffffff   30  ...     69   
64  POINT (104.38128 20.34632)           63  84414e3ffffffff   60  ...     47   
65  POINT (103.88944 19.97709)           47  844148dffffffff  115  ...     35   
66  POINT (104.38128 20.34631)           63  84414e3ffffffff   60  ...     47   

    theo_turbines  theo_pv  index_right_renamed  country  \
0             999     4519                   90    Other   
0             999     4519                   92     Laos   
1            1825     9267                   92     Laos   
2            1324     7132                   94    Other   
2            1324     7132                   92     Laos   
..            ...      ...                  ...      ...   
62           1825     9267                   92     Laos   
63           1825     9267                   92     Laos   
64           1772     9575                   92     Laos   
65           1785     9700                   92     Laos   
66           1772     9575                   92     Laos   

    Vientiane road construction costs  \
0                                 0.0   
0                                 0.0   
1                                 0.0   
2                                 0.0   
2                                 0.0   
..                                ...   
62                                0.0   
63                                0.0   
64                                0.0   
65                                0.0   
66                                0.0   

    Vientiane trucking transport and conversion costs  \
0                                            0.675338   
0                                            0.675338   
1                                            0.641315   
2                                            0.667232   
2                                            0.667232   
..                                                ...   
62                                           0.641315   
63                                           0.641315   
64                                           0.541980   
65                                           0.516568   
66                                           0.541980   

    Vientiane trucking state  \
0                        NH3   
0                        NH3   
1                        NH3   
2                        NH3   
2                        NH3   
..                       ...   
62                       NH3   
63                       NH3   
64                       NH3   
65                       NH3   
66                       NH3   

    Vientiane pipeline transport and conversion costs  p

## Combined code

In [27]:
def hydropower_potential(eta,flowrate,head):
    '''
    Calculate hydropower potential in Megawatts
    eta: Efficiency
    '''
    rho = 997 # kg/m3; Density of water
    g = physical_constants['standard acceleration of gravity'][0] # m/s2; Based on the CODATA constants 2018
    Q = flowrate / 3600 # transform flowrate per h into flowrate per second
    return (eta * rho * g * Q * head) / (1000 * 1000) # MW

def hydropower_potential_with_capacity(flowrate, head, capacity, eta):
    potential = hydropower_potential(flowrate, head, eta)
    limited_potential = xr.where(potential > capacity, capacity, potential)
    capacity_factor = limited_potential / capacity
    return capacity_factor



In [31]:
head = location_hydro['head'].values  # Extract head values
capacity = location_hydro['capacity'].values  # Extract capacity values

# Create DataArrays for head and capacity to align with the plant dimension
head_da = xr.DataArray(head, dims=['plant'])
capacity_da = xr.DataArray(capacity, dims=['plant'])

# Apply the wrapper function using apply_ufunc
capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    hydro_profile,
    head_da,
    capacity_da,
    eta,
    vectorize=True,
    dask='parallelized',  # Optionally, use Dask for parallel computation
    output_dtypes=[float]
)


In [36]:
location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)

# Ensure both GeoDataFrames are in the same CRS
if location_hydro.crs != hexagons.crs:
    location_hydro = location_hydro.to_crs(hexagons.crs)

eta = 0.75 # efficiency of hydropower plant
# Rename existing 'index_left' and 'index_right' columns if they exist
if 'index_left' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_left': 'index_left_renamed'})
if 'index_right' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_right': 'index_right_renamed'})
if 'index_left' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_left': 'index_left_renamed'})
if 'index_right' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_right': 'index_right_renamed'})

# Perform spatial join to map each hydro plant to its corresponding hexagon
hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')

# Add the hydro plant indices to the mapping
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index

# Get the number of hexagons and time steps dynamically
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

# Initialize an empty DataArray for the new capacity_factor with dynamic dimensions
capacity_hexagon = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

# Iterate through each hexagon and calculate the average capacity factor for plants within it
for hex_index in range(num_hexagons):
    # Find the plant indices that are within the current hexagon
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    
    if len(plants_in_hex) > 0:
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        plant_capacities = xr.DataArray(location_hydro.set_index('hexagon').loc[plants_in_hex]['capacity'], dims=['plant'])

        weights = plant_capacities / plant_capacities.sum()
        weighted_avg_capacity_factor = (hex_capacity_factor * weights).sum(dim='hexagon')
        hydro_profile.loc[hex_index] = weighted_avg_capacity_factor

KeyError: "None of ['hexagon'] are in the columns"

In [39]:

location_hydro = gpd.read_file('Data/hydropower_dams.gpkg')
location_hydro.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}, inplace=True)
location_hydro.rename(columns={'head_example':'head'},inplace=True)

laos_hydrobasins = gpd.read_file('hydrobasins_lvl10/hybas_as_lev10_v1c.shp')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x

runoff = cutout.hydro(
    plants=location_hydro,
    hydrobasins= laos_hydrobasins,
    per_unit=True                    # Normalize output per unit area
)

eta = 0.75 # efficiency of hydropower plant

capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    runoff,
    xr.DataArray(location_hydro['head'].values, dims=['plant']),
    xr.DataArray(location_hydro['capacity'].values, dims=['plant']),
    eta,
    vectorize=True,
    dask='parallelized',  # Dask for parallel computation
    output_dtypes=[float]
)

location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)


# Rename existing 'index_left' and 'index_right' columns if they exist
if 'index_left' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_left': 'index_left_renamed'})
if 'index_right' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_right': 'index_right_renamed'})
if 'index_left' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_left': 'index_left_renamed'})
if 'index_right' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_right': 'index_right_renamed'})

hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

hydro_profile = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

Determine upstream basins per plant: 67it [00:02, 23.82it/s]
c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


[########################################] | 100% Completed | 14.89 s


Shift and aggregate runoff by plant: 67it [00:07,  9.49it/s]


In [66]:
hydro_profile

<xarray.DataArray (hexagon: 931, time: 8784)> Size: 65MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * hexagon  (hexagon) int32 4kB 0 1 2 3 4 5 6 7 ... 924 925 926 927 928 929 930
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [68]:
grid_profile = xr.full_like(hydro_profile, fill_value=1)
grid_profile

<xarray.DataArray (hexagon: 931, time: 8784)> Size: 65MB
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])
Coordinates:
  * hexagon  (hexagon) int32 4kB 0 1 2 3 4 5 6 7 ... 924 925 926 927 928 929 930
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [46]:
hydro_hex_mapping[['plant_index','index_right']]

plant_index  index_right
0             0          NaN
1             1        888.0
2             2        239.0
3             3        627.0
4             4        485.0
..          ...          ...
62           62        488.0
63           63        488.0
64           64        422.0
65           65        130.0
66           66        422.0

[71 rows x 2 columns]

In [51]:
location_hydro

lat         lon status   COD  capacity  head  \
0   13.943893  105.955782   COMM  2020   260.000    20   
1   15.543679  106.260687   COMM  1999   152.100    20   
2   15.774865  107.054896   COMM  2023    60.000    20   
3   15.354431  106.161330   COMM  2021     5.000    20   
4   15.356495  106.498949   COMM  2015    84.800    20   
..        ...         ...    ...   ...       ...   ...   
62  15.491744  106.278621   COMM  1994    45.000    20   
63  15.403775  106.280332   COMM  2009    76.000    20   
64  20.346323  104.381276   COMM  2019     9.136    20   
65  19.977092  103.889435   COMM  2019    32.000    20   
66  20.346313  104.381284   COMM  2019     9.100    20   

                      geometry  
0   POINT (105.95578 13.94389)  
1   POINT (106.26069 15.54368)  
2   POINT (107.05490 15.77487)  
3   POINT (106.16133 15.35443)  
4   POINT (106.49895 15.35650)  
..                         ...  
62  POINT (106.27862 15.49174)  
63  POINT (106.28033 15.40377)  
64  POINT (104.38128 20.34632)  
65  POINT (103.88944 19.97709)  
66  POINT (104.38128 20.34631)  

[67 rows x 7 columns]

In [44]:
hydro_hex_mapping.columns

Index(['lat', 'lon', 'status', 'COD', 'capacity', 'head', 'geometry',
       'index_right', 'h3_index', 'n0', 'n1', 'n2', 'n3', 'n4', 'n5',
       'waterbody_dist', 'waterway_dist', 'road_dist', 'hydro', 'index',
       'theo_turbines', 'theo_pv', 'index_right_renamed', 'country',
       'Vientiane road construction costs',
       'Vientiane trucking transport and conversion costs',
       'Vientiane trucking state',
       'Vientiane pipeline transport and conversion costs', 'plant_index'],
      dtype='object')

In [54]:
hexagons = gpd.read_file('Resources/hex_transport.geojson')
hexagons['hydro'] = hexagons['hydro'].fillna(0)

In [62]:
hexagons['Vientiane trucking transport and conversion costs']

0      0.414455
1      0.578949
2      0.673873
3      0.627544
4      0.627544
         ...   
926    0.584484
927    0.531083
928    0.505670
929    0.676506
930    0.669587
Name: Vientiane trucking transport and conversion costs, Length: 931, dtype: float64

In [53]:
for hex_index in range(num_hexagons):
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    if len(plants_in_hex) > 0:
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        plant_capacities = xr.DataArray(location_hydro.loc[plants_in_hex]['capacity'].values, dims=['plant'])

        weights = plant_capacities / plant_capacities.sum()
        weighted_avg_capacity_factor = (hex_capacity_factor * weights).sum(dim='plant')
        hydro_profile.loc[hex_index] = weighted_avg_capacity_factor

In [124]:
# Check if all values for each plant are equal to 1 across the time dimension
all_zeros_per_plant = (capacity_hexagon == 0).all(dim='time')

# Count the number of plants that have all values equal to 1
count_all_zeros = all_zeros_per_plant.sum().item()

# Print the result
print(f"Number of plants with all values equal to 1: {count_all_zeros}")


Number of plants with all values equal to 1: 753


## Test PyPsa

In [3]:
from osgeo import gdal
import atlite
import geopandas as gpd
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import p_H2_aux as aux
from functions import CRF
import numpy as np
import logging
import time

import xarray as xr
from scipy.constants import physical_constants

In [4]:
n = pypsa.Network(override_component_attrs=aux.create_override_components())

In [32]:
transport_excel_path = "Parameters/transport_parameters.xlsx"
weather_excel_path = "Parameters/weather_parameters.xlsx"
country_excel_path = 'Parameters/country_parameters.xlsx'
country_parameters = pd.read_excel(country_excel_path,
                                    index_col='Country')
demand_excel_path = 'Parameters/demand_parameters.xlsx'
demand_parameters = pd.read_excel(demand_excel_path,
                                    index_col='Demand center',
                                    ).squeeze("columns")
demand_centers = demand_parameters.index
weather_parameters = pd.read_excel(weather_excel_path,
                                    index_col = 'Parameters'
                                    ).squeeze('columns')
weather_filename = weather_parameters['Filename']

hexagons = gpd.read_file('Resources/hex_transport.geojson')
# !!! change to name of cutout in weather
cutout = atlite.Cutout('Cutouts/' + weather_filename +'.nc')
layout = cutout.uniform_layout()

###############################################################
# Added for hydropower

location_hydro = gpd.read_file('Data/hydropower_dams.gpkg')
location_hydro.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}, inplace=True)
location_hydro.rename(columns={'head_example':'head'},inplace=True)

laos_hydrobasins = gpd.read_file('hydrobasins_lvl10/hybas_as_lev10_v1c.shp')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x

runoff = cutout.hydro(
    plants=location_hydro,
    hydrobasins= laos_hydrobasins,
    per_unit=True                    # Normalize output per unit area
)

eta = 0.75 # efficiency of hydropower plant

capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    runoff,
    xr.DataArray(location_hydro['head'].values, dims=['plant']),
    xr.DataArray(location_hydro['capacity'].values, dims=['plant']),
    eta,
    vectorize=True,
    dask='parallelized',  # Dask for parallel computation
    output_dtypes=[float]
)

location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)


# Rename existing 'index_left' and 'index_right' columns if they exist
if 'index_left' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_left': 'index_left_old'})
if 'index_right' in location_hydro.columns:
    location_hydro = location_hydro.rename(columns={'index_right': 'index_right_old'})
if 'index_left' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_left': 'index_left_old'})
if 'index_right' in hexagons.columns:
    hexagons = hexagons.rename(columns={'index_right': 'index_right_old'})


hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

hydro_profile = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

for hex_index in range(num_hexagons):
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    if len(plants_in_hex) > 0:
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        average_capacity_factor = hex_capacity_factor.mean(dim='plant')
        hydro_profile.loc[hex_index] = average_capacity_factor

###############################################################

pv_profile = cutout.pv(
    panel= 'CSi',
    orientation='latitude_optimal',
    layout = layout,
    shapes = hexagons,
    per_unit = True
    )
pv_profile = pv_profile.rename(dict(dim_0='hexagon'))

wind_profile = cutout.wind(
    # Changed turbine type - was Vestas_V80_2MW_gridstreamer in first run
    # Other option being explored: NREL_ReferenceTurbine_2020ATB_4MW, Enercon_E126_7500kW
    turbine = 'NREL_ReferenceTurbine_2020ATB_4MW',
    layout = layout,
    shapes = hexagons,
    per_unit = True
    )
wind_profile = wind_profile.rename(dict(dim_0='hexagon'))

Determine upstream basins per plant: 67it [00:02, 24.67it/s]
c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)
INFO:atlite.convert:Convert and aggregate 'runoff'.


[########################################] | 100% Completed | 15.51 s


Shift and aggregate runoff by plant: 67it [00:04, 14.64it/s]
INFO:atlite.convert:Convert and aggregate 'pv'.


[########################################] | 100% Completed | 89.12 s


c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\atlite\resource.py:72: FutureWarning: 'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.13.
  warnings.warn(msg, FutureWarning)
INFO:atlite.convert:Convert and aggregate 'wind'.


[########################################] | 100% Completed | 26.22 s


In [33]:
def optimize_hydrogen_plant(wind_potential, pv_potential, hydro_potential, times, demand_profile,
                            wind_max_capacity, pv_max_capacity, hydro_max_capacity, 
                            country_series, water_limit = None):
    '''
    Optimizes the size of green hydrogen plant components based on renewable potential, hydrogen demand, and country parameters. 

    Parameters
    ----------
    wind_potential : xarray DataArray
        1D dataarray of per-unit wind potential in hexagon.
    pv_potential : xarray DataArray
        1D dataarray of per-unit solar potential in hexagon.
    times : xarray DataArray
        1D dataarray with timestamps for wind and solar potential.
    demand_profile : pandas DataFrame
        hourly dataframe of hydrogen demand in kg.
    country_series : pandas Series
        interest rate and lifetime information.
    water_limit : float
        annual limit on water available for electrolysis in hexagon, in cubic meters. Default is None.

    Returns
    -------
    lcoh : float
        levelized cost per kg hydrogen.
    wind_capacity: float
        optimal wind capacity in MW.
    solar_capacity: float
        optimal solar capacity in MW.
    electrolyzer_capacity: float
        optimal electrolyzer capacity in MW.
    battery_capacity: float
        optimal battery storage capacity in MW/MWh (1 hour batteries).
    h2_storage: float
        optimal hydrogen storage capacity in MWh.

    '''

    # if a water limit is given, check if hydrogen demand can be met
    if water_limit != None:
        # total hydrogen demand in kg
        total_hydrogen_demand = demand_profile['Demand'].sum()
        # check if hydrogen demand can be met based on hexagon water availability
        water_constraint =  total_hydrogen_demand <= water_limit * 111.57 # kg H2 per cubic meter of water
        if water_constraint == False:
            print('Not enough water to meet hydrogen demand!')
            # return null values
            lcoh = np.nan
            wind_capacity = np.nan
            solar_capacity = np.nan
            hydro_capacity = np.nan
            electrolyzer_capacity = np.nan
            battery_capacity = np.nan
            h2_storage = np.nan
            return lcoh, wind_capacity, solar_capacity, hydro_capacity, electrolyzer_capacity, battery_capacity, h2_storage

    # Set up network
    # Import a generic network
    n = pypsa.Network(override_component_attrs=aux.create_override_components())

    # Set the time values for the network
    n.set_snapshots(times)

    # Import the design of the H2 plant into the network
    n.import_from_csv_folder("Parameters/Basic_H2_plant")

    # Import demand profile
    # Note: All flows are in MW or MWh, conversions for hydrogen done using HHVs. Hydrogen HHV = 39.4 MWh/t
    # hydrogen_demand = pd.read_excel(demand_path,index_col = 0) # Excel file in kg hydrogen, convert to MWh
    n.add('Load',
          'Hydrogen demand',
          bus = 'Hydrogen',
          p_set = demand_profile['Demand']/1000*39.4,
          )

    # Send the weather data to the model
    n.generators_t.p_max_pu['Wind'] = wind_potential
    n.generators_t.p_max_pu['Solar'] = pv_potential
    n.generators_t.p_max_pu['Hydro'] = hydro_potential

    # specify maximum capacity based on land use
    n.generators.loc['Wind','p_nom_max'] = wind_max_capacity
    n.generators.loc['Solar','p_nom_max'] = pv_max_capacity 
    n.generators.loc['Hydro','p_nom_max'] = hydro_max_capacity 

    # specify technology-specific and country-specific WACC and lifetime here
    n.generators.loc['Wind','capital_cost'] = n.generators.loc['Wind','capital_cost']\
        * CRF(country_series['Wind interest rate'], country_series['Wind lifetime (years)'])
    n.generators.loc['Solar','capital_cost'] = n.generators.loc['Solar','capital_cost']\
        * CRF(country_series['Solar interest rate'], country_series['Solar lifetime (years)'])
    ### Calculation hydro costs!!! 
    n.generators.loc['Hydro','capital_cost'] = 50
    
    
    for item in [n.links, n.stores,n.storage_units]:
        item.capital_cost = item.capital_cost * CRF(country_series['Plant interest rate'],country_series['Plant lifetime (years)'])

    # Solve the model
    solver = 'gurobi'
    n.lopf(solver_name=solver,
           solver_options = {'LogToConsole':0, 'OutputFlag':0},
           pyomo=False,
           extra_functionality=aux.extra_functionalities,
           )
    # Output results

    lcoh = n.objective/(n.loads_t.p_set.sum()[0]/39.4*1000) # convert back to kg H2
    wind_capacity = n.generators.p_nom_opt['Wind']
    solar_capacity = n.generators.p_nom_opt['Solar']
    hydro_capacity = n.generators.p_nom_opt['Hydro']
    electrolyzer_capacity = n.links.p_nom_opt['Electrolysis']
    battery_capacity = n.storage_units.p_nom_opt['Battery']
    h2_storage = n.stores.e_nom_opt['Compressed H2 Store']
    print(lcoh)
    return lcoh, wind_capacity, solar_capacity, hydro_capacity, electrolyzer_capacity, battery_capacity, h2_storage

In [6]:
location = 'Laos'
lcohs_trucking = np.zeros(len(pv_profile.hexagon))
solar_capacities= np.zeros(len(pv_profile.hexagon))
wind_capacities= np.zeros(len(pv_profile.hexagon))
####################################################
# Added for hydropower
hydro_capacites= np.zeros(len(hydro_profile.hexagon))
####################################################
electrolyzer_capacities= np.zeros(len(pv_profile.hexagon))
battery_capacities = np.zeros(len(pv_profile.hexagon))
h2_storages= np.zeros(len(pv_profile.hexagon))
start = time.process_time()
# function
for hexagon in pv_profile.hexagon.data:
    hydrogen_demand_trucking, hydrogen_demand_pipeline = demand_schedule(
        demand_parameters.loc[location,'Annual demand [kg/a]'],
        hexagons.loc[hexagon,f'{location} trucking state'],
        transport_excel_path,
        weather_excel_path)
    country_series = country_parameters.loc[hexagons.country[hexagon]]
    lcoh, wind_capacity, solar_capacity, electrolyzer_capacity, battery_capacity, h2_storage =\
        optimize_hydrogen_plant(wind_profile.sel(hexagon = hexagon),
                            pv_profile.sel(hexagon = hexagon),
                            wind_profile.time,
                            hydrogen_demand_trucking,
                            hexagons.loc[hexagon,'theo_turbines'],
                            hexagons.loc[hexagon,'theo_pv'],
                            country_series,
                            # water_limit = hexagons.loc[hexagon,'delta_water_m3']
                            )
    lcohs_trucking[hexagon] = lcoh
    solar_capacities[hexagon] = solar_capacity
    wind_capacities[hexagon] = wind_capacity
    electrolyzer_capacities[hexagon] = electrolyzer_capacity
    battery_capacities[hexagon] = battery_capacity
    h2_storages[hexagon] = h2_storage

NameError: name 'times' is not defined

In [39]:
1.58E+06 * CRF(0.06, 20)

137751.6000234252

In [41]:
1900000 * CRF(0.06, 20)

165650.65825601766